In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
# from dotenv import load_dotenv
import os
import json
import pymupdf
import tempfile
from tqdm import tqdm
import sqlite3
from functions import *

## Filtrando sentenças até a primeira instância

In [ ]:
anexos = pd.read_excel('docs/jusbrasil/jusbrasil.xlsx', sheet_name="Anexos")

In [44]:
anexos_copia = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Publicado em"]]
sentencas = anexos_copia.loc[anexos["Tipo de anexo"].isin(["SENTENCA"]),
                                           ["processoID", "processoAnexoID", "Download copia", "Publicado em"]]
sentencas.head()

In [45]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = sentencas[sentencas['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)
acessos_negados.head()

In [ ]:
# Dropando os acessos negados
# sentencas_acessaveis = sentencas[sentencas["processoAnexoID"].isin(acessos_negados["processoAnexoID"]) == False]

# Exportando sentenças sem acesso negado para um arquivo Excel
# sentencas_acessaveis.to_excel("docs/jusbrasil/sentencas_acessaveis.xlsx", index=False)
sentencas_acessaveis = pd.read_excel("docs/jusbrasil/sentencas_acessaveis.xlsx")

In [47]:
# Verificando quantidade de processos após o drop
sentencas_acessaveis["processoID"].unique().shape

In [51]:
ids_processos = sentencas_acessaveis["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

In [52]:
sentencas_final = sentencas_acessaveis.copy()
for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        if linhas_correspondentes['Publicado em'].nunique() == 1:
            # Data de publicação igual: eliminando por ordem de anexo id
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["processoAnexoID"], ascending=True)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        else:
            # Ordenando as linhas por data
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["Publicado em"], ascending=True)
            # Mantendo a primeira linha (mais antiga)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        
        # Eliminar todas as outras com mesmo processoID e processoAnexoID diferente do que foi mantido
        sentencas_final = sentencas_final[~((sentencas_final["processoID"] == id) & (sentencas_final["processoAnexoID"] != manter_id))]

In [53]:
# Verificando dataframe após a eliminação
ids_processos = sentencas_final["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_final.loc[sentencas_final['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

In [54]:
# Verificando se a quantidade de processos após o drop se manteve
sentencas_final["processoID"].unique().shape

## Extraindo dados das sentenças

In [ ]:
# Exportando df final de sentenças para um arquivo Excel
# sentencas_final.to_excel("docs/jusbrasil/sentencas_final.xlsx", index=False)
sentencas_final = pd.read_excel("docs/jusbrasil/sentencas_final.xlsx")

### Prompt 1 - Filtro: trata-se de um caso de dano ambiental?

In [30]:
respostas = []
ids_processos = sentencas_final["processoID"].unique()

for id in tqdm(ids_processos):
    linha = sentencas_final.loc[sentencas_final['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['Download copia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = verifica_dano_ambiental(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas.append(resposta_prompt)

respostas

In [31]:
# Transformando a lista de respostas em um DataFrame
respostas_df = pd.DataFrame(respostas)

# Reorganizando as colunas
respostas_df = respostas_df[["processoID", "processoAnexoID", "isDanoAmbiental", "justificativa", "link_referencia"]]

respostas_df.head()

In [2]:
# Exportando df de classificação de sentenças para um arquivo Excel
# respostas_df.to_excel("docs/jusbrasil/respostas_classificacao_sentencas.xlsx", index=False)
respostas_df = pd.read_excel("docs/jusbrasil/respostas_classificacao_sentencas.xlsx")

In [3]:
sentencas_danos_ambientais = respostas_df.loc[respostas_df["isDanoAmbiental"] == True]

### Prompt 2 - Extração dos dados

In [4]:
lista_processos = list(sentencas_danos_ambientais['processoID'].unique())

partes = divide_lista_em_partes(lista_processos, 4)
print("Distribuição de processos entre as partes:")
for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
print("Total de processos:", len(lista_processos))

Distribuição de processos entre as partes:
Parte 1: 19 processos
Parte 2: 19 processos
Parte 3: 19 processos
Parte 4: 22 processos
Total de processos: 79


In [38]:
# Analisando os processos com dano ambiental
respostas_danos_ambientais = []
for id in tqdm(lista_processos):
    linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['link_referencia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = analisa_sentenca(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas_danos_ambientais.append(resposta_prompt)

respostas_danos_ambientais

In [41]:
# Transformando a lista de respostas de danos ambientais da primeira parte em um DataFrame
respostas_danos_ambientais_df_completo = pd.DataFrame(respostas_danos_ambientais)

# Reorganizando as colunas
respostas_danos_ambientais_df_completo = respostas_danos_ambientais_df_completo[["numero_processo", "processoID", "processoAnexoID", "georreferencia", "uf", "municipio", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "link_referencia"]]

respostas_danos_ambientais_df_completo.head()

In [ ]:
# respostas_danos_ambientais_df_completo.to_excel("docs/jusbrasil/respostas_danos_ambientais_df_completo.xlsx", index=False)

## Juntando JusBrasil e IOPC

In [2]:
respostas_danos_ambientais_df_completo = pd.read_excel("docs/jusbrasil/respostas_danos_ambientais_df_completo.xlsx")
df_iopc = pd.read_excel("docs/iopc/iopc_tables_final.xlsx")

In [3]:
# Coluna de moeda toda preenchida com R$
respostas_danos_ambientais_df_completo['moeda'] = 'R$'
respostas_danos_ambientais_df_completo['fonte_dados'] = 'JusBrasil'
respostas_danos_ambientais_df_completo['pais'] = 'Brasil'
# Ordenando as colunas
respostas_danos_ambientais_df_completo = respostas_danos_ambientais_df_completo[["fonte_dados", "numero_processo", "processoID", "processoAnexoID", "georreferencia", "pais", "uf", "municipio", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "moeda", "link_referencia"]]
respostas_danos_ambientais_df_completo.columns

In [4]:
df_iopc["fonte_dados"] = "IOPC"
df_iopc.rename(columns={"Date of Incident": "data_impacto", 
                        "Currency": "moeda", 
                        "Compensation": "valor_multa", 
                        "Estimated quantity of oil spilled (tonnes)": "qtde_petroleo_derramada"}, inplace=True)

# Split da coluna Place of Incident em pais e região. Se não encontrar vírgula, coloca NULL em regiao e a informação completa em pais
if df_iopc['Place of Incident'].str.contains(',').any():
    df_iopc[['regiao', 'pais']] = df_iopc['Place of Incident'].str.split(',', expand=True)
else:
    df_iopc['regiao'] = None
    df_iopc['pais'] = df_iopc['Place of Incident']

# Ordenando as colunas
df_iopc = df_iopc[["fonte_dados", "pais", "regiao", "data_impacto", "qtde_petroleo_derramada", "valor_multa", "moeda"]]
df_iopc.columns

In [5]:
df_iopc

In [6]:
# Juntando os dois DataFrames
df_jusbrasil_iopc = pd.concat([respostas_danos_ambientais_df_completo, df_iopc], ignore_index=True)

In [7]:
# Reordenando as colunas finais
df_jusbrasil_iopc = df_jusbrasil_iopc[["fonte_dados", "numero_processo", "processoID", "processoAnexoID", "georreferencia", "pais", "uf", "municipio", "regiao", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "moeda", "link_referencia"]]
df_jusbrasil_iopc.fillna('NULL', inplace=True)
df_jusbrasil_iopc

In [ ]:
# df_jusbrasil_iopc.to_excel("docs/df_jusbrasil_iopc.xlsx", index=False)

## Juntando com o Juscraper

In [13]:
df_juscraper = pd.read_excel("docs/juscraper/respostas_danos_ambientais_juscraper.xlsx")
df_juscraper["fonte_dados"] = "Juscraper"
df_juscraper['moeda'] = 'R$'
df_juscraper['pais'] = 'Brasil'
df_jusbrasil_iopc_juscraper = pd.concat([df_jusbrasil_iopc, df_juscraper], ignore_index=True)

In [15]:
df_jusbrasil_iopc_juscraper = df_jusbrasil_iopc_juscraper[["fonte_dados", "numero_processo", "processoID", "processoAnexoID", "georreferencia", "pais", "uf", "municipio", "regiao", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "moeda", "link_referencia", "referencia"]]
df_jusbrasil_iopc_juscraper.fillna('NULL', inplace=True)
df_jusbrasil_iopc_juscraper

In [31]:
# Verificando se há processos duplicados entre jusbrasil e juscraper
df_jusbrasil_juscraper = df_jusbrasil_iopc_juscraper.loc[df_jusbrasil_iopc_juscraper['fonte_dados'] != 'IOPC']

for num in df_jusbrasil_juscraper['numero_processo'].unique().tolist():
    filtro = df_jusbrasil_juscraper.loc[df_jusbrasil_juscraper['numero_processo'] == num]
    if filtro.shape[0] > 1:
        display(filtro)
        print('*'*100)

print(len(df_jusbrasil_juscraper['numero_processo'].tolist()))
print(len(df_jusbrasil_juscraper['numero_processo'].unique().tolist()))

In [ ]:
# df_jusbrasil_iopc_juscraper.to_excel("docs/df_jusbrasil_iopc_juscraper.xlsx", index=False)

## Extracao Nasser

In [13]:
# --- CONFIGURAÇÕES DO BANCO DE DADOS SQLITE ---
DB_FILE_NAME = "meu_banco_gemini.db"
TABLE_PROCESSOS = "processos_analisados_gemini"

# Colunas na ordem exata que você especificou para o DataFrame final
# Estas são as colunas que você espera que a API Gemini retorne (ou que você preencha)
# mais as colunas 'processoID', 'processoAnexoID', 'link_referencia'
COLUNAS_DATAFRAME_FINAL = [
    "numero_processo", "processoID", "processoAnexoID", "georreferencia", "uf", "municipio",
    "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto",
    "data_impacto", "area_afetada", "unidade_area", "houve_compensacao",
    "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria",
    "link_referencia"
]

In [14]:
# --- FUNÇÕES AUXILIARES SQLITE ---

def criar_tabela_sqlite():
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        # Criar colunas com tipo TEXT para flexibilidade. processoID será a chave primária.
        colunas_sql = ", ".join([f'"{col}" TEXT' for col in COLUNAS_DATAFRAME_FINAL if col != "processoID"])
        # processoID é especial e usado como PRIMARY KEY
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_PROCESSOS} (
            "processoID" TEXT PRIMARY KEY,
            {colunas_sql}
        )
        """)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Erro ao criar/conectar tabela SQLite: {e}")
    finally:
        if conn:
            conn.close()

def get_ids_processados_sqlite():
    conn = None
    ids_processados = set()
    # Verificar se o arquivo do banco de dados existe antes de tentar conectar
    if not os.path.exists(DB_FILE_NAME):
        criar_tabela_sqlite() # Cria a tabela se o DB não existir
        return ids_processados

    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        # Verifica se a tabela existe antes de tentar selecionar
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_PROCESSOS}';")
        if cursor.fetchone() is None:
            # Tabela não existe, então crie-a
            conn.close() # Fecha a conexão atual para evitar problemas
            criar_tabela_sqlite()
            return ids_processados # Retorna conjunto vazio pois nada foi processado

        cursor.execute(f"SELECT processoID FROM {TABLE_PROCESSOS}")
        rows = cursor.fetchall()
        # Os IDs no seu DataFrame e lista_processos podem ser int ou str.
        # Para consistência, vamos tratar como string ao comparar.
        ids_processados = {str(row[0]) for row in rows}
    except sqlite3.Error as e:
        print(f"Erro ao ler IDs do SQLite: {e}")
    finally:
        if conn:
            conn.close()
    return ids_processados

def inserir_dados_processo_sqlite(dados_dict):
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        
        # Garantir que todos os campos de COLUNAS_DATAFRAME_FINAL existam no dict, preenchendo com None se faltar
        # e convertendo todos os valores para string para inserção segura como TEXT
        valores_ordenados = []
        for col_nome in COLUNAS_DATAFRAME_FINAL:
            valor = dados_dict.get(col_nome)
            valores_ordenados.append(str(valor) if valor is not None else None)

        cols_string = ", ".join([f'"{col}"' for col in COLUNAS_DATAFRAME_FINAL])
        placeholders = ", ".join(["?"] * len(COLUNAS_DATAFRAME_FINAL))
        
        # Usar INSERT OR REPLACE para atualizar caso já exista (embora a lógica de pular já deva cobrir)
        # ou INSERT OR IGNORE para simplesmente pular se já existir.
        # Como estamos checando `ids_processados` antes, um INSERT simples seria suficiente,
        # mas OR IGNORE é mais seguro contra race conditions ou lógicas imperfeitas.
        cursor.execute(f"INSERT OR REPLACE INTO {TABLE_PROCESSOS} ({cols_string}) VALUES ({placeholders})",
                       valores_ordenados)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Erro ao inserir dados no SQLite para processoID {dados_dict.get('processoID')}: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

def limpar_resposta_json_gemini(texto_json_bruto: str) -> str:
    """Remove marcadores ```json ... ``` de respostas da API Gemini."""
    limpo = texto_json_bruto.strip()
    if limpo.startswith("```json"):
        limpo = limpo[7:]
    elif limpo.startswith("```"):
        limpo = limpo[3:]
    if limpo.endswith("```"):
        limpo = limpo[:-3]
    return limpo.strip()

In [15]:
# Garante que a tabela exista antes de começar
criar_tabela_sqlite()

In [16]:
ids_ja_processados_no_db = get_ids_processados_sqlite()
print(f"Total de IDs na lista original: {len(lista_processos)}")
print(f"IDs já processados e salvos no banco de dados: {len(ids_ja_processados_no_db)}")

# Filtra a lista_processos para rodar apenas os que ainda não estão no DB
# Convertendo os IDs da lista_processos para string para comparação consistente
lista_processos_pendentes = [str(pid) for pid in lista_processos if str(pid) not in ids_ja_processados_no_db]

print(f"IDs pendentes para processamento: {len(lista_processos_pendentes)}")

respostas_danos_ambientais = [] # Sua lista original para acumular resultados da rodada atual


In [ ]:
if not lista_processos_pendentes:
    print("Todos os processos já foram analisados e estão no banco de dados.")
else:
    for id_str in tqdm(lista_processos_pendentes, desc="Analisando processos pendentes"):
        # O 'id' original pode ser int ou str. Vamos garantir que estamos pegando do DataFrame corretamente.
        # Tentamos como string primeiro, depois como int se o DataFrame usar int.
        try:
            linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'].astype(str) == id_str]
        except KeyError: # Se a coluna 'processoID' não for string e falhar a conversão/comparação
             # Tenta converter o id_str para o tipo da coluna 'processoID' no DataFrame
            id_original_tipo = type(sentencas_danos_ambientais['processoID'].iloc[0])
            try:
                id_convertido = id_original_tipo(id_str)
                linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'] == id_convertido]
            except Exception as e_conv:
                print(f"  AVISO: Não foi possível encontrar o processoID '{id_str}' (ou convertido) no DataFrame 'sentencas_danos_ambientais'. Erro de conversão: {e_conv}. Pulando.")
                continue

        if linha.empty:
            print(f"  AVISO: ProcessoID '{id_str}' não encontrado em 'sentencas_danos_ambientais'. Pulando.")
            continue

        texto_extraido = ''
        linha_serie = linha.iloc[0] # extrai a linha como Series

        processoAnexoID = linha_serie['processoAnexoID']
        link = linha_serie['link_referencia']
        
        print(f"\nProcessando ID: {id_str} | AnexoID: {processoAnexoID} | Link: {link}")

        try:
            response = requests.get(link, timeout=60) # Timeout maior para downloads
            response.raise_for_status() # Levanta exceção para erros HTTP 4xx/5xx
            response.encoding = response.apparent_encoding if response.apparent_encoding else 'utf-8' # Melhor detecção de encoding
            content_type = response.headers.get('Content-Type', '').lower()
            
            if 'pdf' in content_type:
                # print("  Extraindo texto de PDF...")
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                    tmp_file.write(response.content)
                    tmp_path = tmp_file.name
                try:
                    with pymupdf.open(tmp_path) as doc:
                        for page_num, page in enumerate(doc):
                            texto_extraido += page.get_text("text") # "text" para melhor extração
                except Exception as e_pdf:
                    print(f"  ERRO ao processar PDF de {link}: {e_pdf}. Pulando este processo.")
                    if os.path.exists(tmp_path): os.remove(tmp_path)
                    continue # Pula para o próximo ID
                finally:
                    if os.path.exists(tmp_path): os.remove(tmp_path)
            
            elif 'html' in content_type or 'text/plain' in content_type or not content_type : # Tenta HTML ou texto puro
                # print("  Extraindo texto de HTML/TEXT...")
                soup = BeautifulSoup(response.content, 'html.parser')
                # Remove tags de script e style
                for script_or_style in soup(["script", "style"]):
                    script_or_style.decompose()
                texto_extraido += soup.get_text(separator='\n', strip=True)
            
            else: # Se não for PDF nem HTML/TEXT conhecido, tenta ler como texto simples
                print(f"  AVISO: Content-Type '{content_type}' não é PDF nem HTML. Tentando ler como texto direto.")
                texto_extraido += response.text

            if not texto_extraido.strip():
                print(f"  AVISO: Nenhum texto foi extraído de {link}. Pulando análise Gemini.")
                continue

            # print(f"  Texto extraído (primeiros 100 chars): {texto_extraido[:100].replace(chr(10), ' ')}...")
            
            # Chamada à sua função que interage com a API Gemini
            # Esta função deve retornar um objeto com um atributo .text contendo o JSON
            resposta_api_obj = analisa_sentenca(texto_extraido) # SUA FUNÇÃO REAL AQUI

            # Limpa e parseia o JSON da resposta
            json_string_limpo = limpar_resposta_json_gemini(resposta_api_obj.text)
            dados_da_api = json.loads(json_string_limpo)
            
            # Monta o dicionário completo para este processo
            # Garante que o 'processoID' seja o mesmo usado para busca (string)
            dados_completos_processo = {"processoID": str(id_str)} 
            dados_completos_processo.update(dados_da_api) # Adiciona dados da API
            
            # Adiciona/sobrescreve as informações que você gerencia manualmente
            dados_completos_processo["processoAnexoID"] = processoAnexoID
            dados_completos_processo["link_referencia"] = link
            
            # Adiciona à lista da rodada atual (como no seu código original)
            respostas_danos_ambientais.append(dados_completos_processo)
            
            # Salva no banco de dados SQLite IMEDIATAMENTE
            inserir_dados_processo_sqlite(dados_completos_processo)
            print(f"  Processo {id_str} analisado e salvo no banco de dados.")

        except requests.exceptions.RequestException as e_req:
            print(f"  ERRO DE REQUEST ao acessar {link}: {e_req}. Pulando este processo.")
        except json.JSONDecodeError as e_json:
            print(f"  ERRO AO DECODIFICAR JSON da API para o processo {id_str}. Resposta: {getattr(resposta_api_obj, 'text', 'N/A')[:200]}. Erro: {e_json}. Pulando.")
        except AttributeError as e_attr: # Caso analisa_sentenca não retorne .text
            print(f"  ERRO: A função 'analisa_sentenca' não retornou um objeto com atributo '.text' para o processo {id_str}. Erro: {e_attr}. Pulando.")
        except Exception as e_geral:
            print(f"  ERRO INESPERADO ao processar ID {id_str}: {e_geral}. Pulando este processo.")
            # import traceback
            # print(traceback.format_exc()) # Para debug mais detalhado se necessário
    
    print("\nProcessamento dos IDs pendentes concluído.")

In [ ]:
print("\nCarregando todos os dados do banco de dados para o DataFrame final...")
conn = None
try:
    conn = sqlite3.connect(DB_FILE_NAME)
    # Lê todas as colunas na ordem definida em COLUNAS_DATAFRAME_FINAL
    # Assegura que todas as colunas sejam lidas, mesmo que algumas estejam vazias para alguns registros
    col_select_str = ", ".join([f'"{col}"' for col in COLUNAS_DATAFRAME_FINAL])
    
    # Verifica se a tabela existe antes de tentar ler
    cursor_check = conn.cursor()
    cursor_check.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_PROCESSOS}';")
    if cursor_check.fetchone() is None:
        print(f"A tabela {TABLE_PROCESSOS} não existe no banco de dados. Nenhum dado para carregar.")
        respostas_danos_ambientais_df_completo = pd.DataFrame(columns=COLUNAS_DATAFRAME_FINAL)
    else:
        respostas_danos_ambientais_df_completo = pd.read_sql_query(f"SELECT {col_select_str} FROM {TABLE_PROCESSOS}", conn)
except sqlite3.Error as e:
    print(f"Erro ao ler dados do SQLite para o DataFrame: {e}")
    respostas_danos_ambientais_df_completo = pd.DataFrame(columns=COLUNAS_DATAFRAME_FINAL) # Cria DF vazio em caso de erro
finally:
    if conn:
        conn.close()

if not respostas_danos_ambientais_df_completo.empty:
    # A ordem das colunas já deve estar correta devido ao SELECT explícito,
    # mas podemos reconfirmar para garantir, caso a leitura do SQL não preserve 100% a ordem.
    respostas_danos_ambientais_df_completo = respostas_danos_ambientais_df_completo[COLUNAS_DATAFRAME_FINAL]
    
    print(f"\nDataFrame final construído a partir do SQLite com {len(respostas_danos_ambientais_df_completo)} registros.")
    print(respostas_danos_ambientais_df_completo.head())

    output_excel_file = "docs/jusbrasil/respostas_danos_ambientais_df_completo.xlsx"
    os.makedirs("docs", exist_ok=True) # Cria o diretório 'docs' se não existir
    respostas_danos_ambientais_df_completo.to_excel(output_excel_file, index=False)
    print(f"\nDataFrame final salvo em: {output_excel_file}")
else:
    print("\nNenhum dado foi carregado do banco de dados para gerar o arquivo Excel.")

# Padronizacao Tipo de Impacto

In [7]:
df_jusbrasil_iopc_juscraper = pd.read_excel("docs\df_jusbrasil_iopc_juscraper.xlsx")
lista_impactos_especificos = df_jusbrasil_iopc_juscraper["tipo_impacto"].tolist()

In [8]:
categorias_generalizadas = [
    'Poluição Hídrica',
    'Poluição do Solo',
    'Poluição do Ar e Sonora',
    'Desmatamento e Danos à Flora',
    'Incêndios e Queimadas',
    'Danos à Fauna',
    'Gestão Inadequada de Resíduos',
    'Ocupação e Construção Irregular',
    'Erosão, Assoreamento e Impactos Geológicos',
    'Extração Ilegal de Recursos Naturais',
    'Falhas e Riscos de Infraestrutura',
    'Impactos Sociais e à Saúde Pública',
    'Danos ao Patrimônio e Bens Públicos',
    'Infrações Administrativas e Legais',
    'Dano Ambiental Genérico / Outros'
]